In [1]:
#ProgramName: Daily-07-Update-Min-Max-DATE-Reporting-incomplete
#Purpose: Run collection routine for min-max details about system (Quality Assurance)
#Author:  Greg Turmel, Director, Data Governance 
#Date:    2020.08.30 - 2021.06.30
#Errata:  0.1 Improvements can be made to script using for/looping through the databases

import os, sys, argparse, pyodbc, sql, time, datetime
import errno, pathlib2
import sqlalchemy as db
import errno, pathlib2

from dotenv import load_dotenv # add this line
import pandas as pd
import numpy as np

from io import StringIO
import csv
import re

load_dotenv() # add this line
user = os.getenv('MySQLeUser')
password = os.getenv('MySQLeUserPass')
host = os.getenv('MySQLeHOST')
db = os.getenv('MySQLeDB')

In [2]:
# This segment builds the appropriatte file system structure as a variable driven exercise
# Take time to set your 'eeeeeeeeee' number as variable 'pn' below 
# and set the program directory variable called 'programDirectory' before running
# ===================================================================================================================
pn = r'eeeeeeeeee'            #This represents the windows system employee login folder - IBM team uses a 9 number
# ===================================================================================================================

programDirectory = 'Daily' # Update this variable to wherever you want the program subfolder/files to be located 
un = r'C:\Users'
cn = r'Documents\Py'
an = r'MinMax\Process'
bn = r'MinMax\Reports'

sn = r'MinMax\sql'
tn = r'config'

#Set a parent directory
parentDirectory = "{}\{}\{}".format(un,pn,cn)
print('Parent Directory is: ', parentDirectory)
mode = 0o666

#Set path location for working with local file(s)
path = os.path.join(parentDirectory, programDirectory,)
pathMP = os.path.join(parentDirectory, programDirectory, an)
pathMR = os.path.join(parentDirectory, programDirectory, bn)
pathMS = os.path.join(parentDirectory, programDirectory, sn)
pathMT = os.path.join(parentDirectory, programDirectory, tn)

procpath = pathMP
csvpath = pathMR
sqlpath = pathMS
configpath = pathMT

try:
    if not os.path.exists(path):
        os.makedirs(path, mode)
        print('Program Directory subfolder has been created: ', programDirectory)
    else:
        print('Directory:', programDirectory, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMP):
        os.makedirs(pathMP, mode)
        print('Program Directory subfolder has been created: ', pathMP)
    else:
        print('Directory:', pathMP, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMR):
        os.makedirs(pathMR, mode)
        print('Program Directory subfolder has been created: ', pathMR)
    else:
        print('Directory:', pathMR, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMS):
        os.makedirs(pathMS, mode)
        print('Program Directory subfolder has been created: ', pathMS)
    else:
        print('Directory:', pathMS, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMT):
        os.makedirs(pathMT, mode)
        print('Program Directory subfolder has been created: ', pathMT)
    else:
        print('Central Config Directory:', pathMT, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

Parent Directory is:  C:\Users\e201873842\Documents\Py
Directory: Daily >>>>> Note ---- this folder already exists <<<<<
Program Directory subfolder has been created:  C:\Users\e201873842\Documents\Py\Daily\MinMax\Process
Directory: C:\Users\e201873842\Documents\Py\Daily\MinMax\Reports >>>>> Note ---- this folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\MinMax\sql >>>>> Note ---- this folder already exists <<<<<
Central Config Directory: C:\Users\e201873842\Documents\Py\Daily\config >>>>> Note ---- this folder already exists <<<<<


In [4]:
# ===================================================================================================================
# Variables: Program set up
# ===================================================================================================================
#0. Set and specify a file name for each variable 
#1. Establish if prod or test run using csvtype, debugging using Debug variable, and set time variable to track process
#2. Read into dataframe and establish database names, schema names, and table names 
#3. For debugging issues: Write dataframe results into an output csv 
#4. Validate database is available today and then create dynamic sql file(s) for processing
now01 = datetime.datetime.now()
csvtype = 'Base' #csvtype - use 'Test or Base'

#Locate/Relocate the driving xls/csv core-noncore file structure based on your system
#Then: Read the excel file(s) with core and noncore source tables 
#(control input using csvtype variable "base" and "test") 
f1a = pd.read_csv("{}/{}".format(pathMT, 'coreSource'+ csvtype +'.csv'), encoding='1252')
f1b = pd.read_csv("{}/{}".format(pathMT, 'noncoreSource'+ csvtype +'.csv'), encoding='1252')

#Set Core dataframes
f1c = f1a['DatabaseName']           #(Core) Create dataframe with what is found in f1 dataframe
f1d = f1a['TableName']              #(Core) Create new dataframe with tables found in f1 dataframe
f1g = f1a['SchemaName']             #(Core) Create new dataframe with tables found in f1 dataframe

#Set NONCore dataframes
f1e = f1b['DatabaseName']           #(Non) Create dataframe with what is found in f1 dataframe
f1f = f1b['TableName']              #(Non) Create new dataframe with tables found in f1 dataframe
f1h = f1b['SchemaName']             #(Non) Create new dataframe with tables found in f1 dataframe

#De-Dupe Core dataframe contents
dbListC = f1c.drop_duplicates()     #de-dupe
tblListC = f1d.drop_duplicates()    #de-dupe
schemaListC = f1g.drop_duplicates() #de-dupe

#Sort Core dataframe contents
dbListC = dbListC.sort_values()            #Sort
tblListC = tblListC.sort_values()          #Sort
schemaListC = schemaListC.sort_values()    #Sort

#De-Dupe NONCore dataframe contents
dbListN = f1e.drop_duplicates()     #de-dupe
tblListN = f1f.drop_duplicates()    #de-dupe
schemaListN = f1h.drop_duplicates() #de-dupe

#Sort NONCore dataframe contents
dbListN = dbListN.sort_values()            #Sort
tblListN = tblListN.sort_values()          #Sort
schemaListN = schemaListN.sort_values()    #Sort

#Debug Control
Dbug = 'Off'     #Debugging - use On or Off  
if Dbug == 'On':
    print (dbListC)
    print (dbListN)
    #break

#Next, Set and specify a path variable for the reports: 

#Set "path" variable used throughout for file creation and data loading 
#Set variable names for output files: 1. debugging dataframes, 2. dynamic sql generated files, and 3. reports to build
file00 = 'dbListC.csv'
file01 = 'dbListN.csv'
file02 = 'tblListC.csv'
file03 = 'tblListN.csv'
file13 = 'schemaListC.csv'
file14 = 'schemaListN.csv'
# ===================================================================================================================
# Setup Dynamic query seperation: CORE
# USE 'CD' for core DATE, 'CI' for core INT, USE 'CC' for core CHAR, VARCHAR, NCHAR, NVARCHAR 
# ===================================================================================================================
file04 = 'mmDynamicCD.sql'
file05 = 'mmDynamicNCD.sql'
file06 = 'mmReport.csv'
# ===================================================================================================================
file10 = 'mmMergeC.csv'
file11 = 'mmMergeN.csv'
file12 = 'mmMerge.csv'
# ===================================================================================================================
# Specify a path above for reports: Make sure output location is available for write 
# ===================================================================================================================
with open(os.path.join(pathMP, file00), 'w') as fp: 
    pass
with open(os.path.join(pathMP, file01), 'w') as fp: 
    pass
with open(os.path.join(pathMP, file02), 'w') as fp: 
    pass
with open(os.path.join(pathMP, file03), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file04), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05), 'w') as fp: 
    pass
with open(os.path.join(pathMR, file06), 'w') as fp: 
    pass
with open(os.path.join(pathMR, file10), 'w') as fp: 
    pass
with open(os.path.join(pathMR, file11), 'w') as fp: 
    pass
with open(os.path.join(pathMR, file12), 'w') as fp: 
    pass
with open(os.path.join(pathMP, file13), 'w') as fp: 
    pass
with open(os.path.join(pathMP, file14), 'w') as fp: 
    pass
# ===================================================================================================================
#  Back up the *dbList, *tblList, *schemaList dataframes from input file for debug
# ===================================================================================================================
#dbList files
with open(os.path.join(pathMP, file00), 'a') as f:
    dbListC.to_csv(f, header=None, index=False, line_terminator='\n')
with open(os.path.join(pathMP, file01), 'a') as f:
    dbListN.to_csv(f, header=None, index=False, line_terminator='\n')

#tblList files
with open(os.path.join(pathMP, file02), 'a') as f:
    tblListC.to_csv(f, header=None, index=False, line_terminator='\n')
with open(os.path.join(pathMP, file03), 'a') as f:
    tblListN.to_csv(f, header=None, index=False, line_terminator='\n')

#schemaList files
with open(os.path.join(pathMP, file13), 'a') as f:
    schemaListC.to_csv(f, header=None, index=False, line_terminator='\n')
with open(os.path.join(pathMP, file14), 'a') as f:
    schemaListN.to_csv(f, header=None, index=False, line_terminator='\n')
# ===================================================================================================================
#  MinMax (DATE) Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
for y in schemaListC:
    try:
        for x in dbListC:
            try:
                conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))

                sql = open(os.path.join(pathMS, 'mmCD.sql'))
                df = pd.read_sql_query(sql.read(), conn)

                with open(os.path.join(pathMS, file04), 'a') as f:
                    df.to_string(f, header=None, index=False)

                sql.close()
                conn.close()
            finally:
                continue
    finally:
        continue

# ===================================================================================================================
#  MinMax (DATE) Core Processing of the dynamic SQL file
# ===================================================================================================================
# Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicCD.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
now01 = datetime.datetime.now()
print('1 Processing Dynamic Min-Max C DATE SQL: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

for sqlCmd in sqlFile.split(';'):
    for y in schemaListC:
        try:
            for x in dbListC:
                try:
                    conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))
                    cursor = conn.cursor()
                    cursor.execute(sqlCmd)
                    row = cursor.fetchone()
                    if row == None:
                        break
                    df = pd.DataFrame([row])
                    #if df == None:
                        #break
                    with open(os.path.join(pathMR, file06), 'a') as f:
                        df.to_csv(f, header=None, index=False, line_terminator='\n')
                    with open(os.path.join(pathMR, file10), 'a') as f:
                        df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
                    with open(os.path.join(pathMR, file12), 'a') as f:
                        df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
                    # Close cursor to database
                    cursor.close()
                    # Close connection to database
                    conn.close()
                except:
                    continue
                finally:
                    continue
        finally:
            continue

now01 = datetime.datetime.now()
print('2 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")
# ===================================================================================================================
#  MinMax (DATE) NonCore - Create Dynamic SQL - Seperating NONcore results from reporting
# ===================================================================================================================
for x in dbListN:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql = open(os.path.join(pathMS, 'mmNCD.sql'))
        df = pd.read_sql_query(sql.read(), conn)

        with open(os.path.join(pathMS, file05), 'a') as f:
            df.to_string(f, header=None, index=False)

        sql.close()
        conn.close()
    finally:
        continue

# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicNCD.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
now01 = datetime.datetime.now()
print('3 Processing Dynamic Min-Max NC DATE SQL: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

for sqlCmd in sqlFile.split(';'):
    for x in dbListN:
        try:
            conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))
            cursor = conn.cursor()
            cursor.execute(sqlCmd)
            row = cursor.fetchone()
            if row == None:
                break
            df = pd.DataFrame([row])
            #if df == None:
                #break
            with open(os.path.join(pathMR, file06), 'a') as f:
                df.to_csv(f, header=None, index=False, line_terminator='\n')
            with open(os.path.join(pathMR, file11), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            with open(os.path.join(pathMR, file12), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            #Close cursor to database
            cursor.close()
            #Close connection to database
            conn.close()
        except:
            continue
        finally:
            continue

# ===================================================================================================================
now01 = datetime.datetime.now()
print('4 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5 **** Connection Closed - Report Generated - Program Completed ****')

1 Processing Dynamic Min-Max C DATE SQL:  2021-05-25 08:16:48
2 Processing Complete:  2021-05-25 08:16:58
3 Processing Dynamic Min-Max NC DATE SQL:  2021-05-25 08:17:17
4 Processing Complete:  2021-05-25 09:02:19
5 **** Connection Closed - Report Generated - Program Completed ****


In [5]:
#Core Type 
#Variables: 
now = datetime.datetime.now()
csvtype = 'Base' #csvtype - use 'Test or Base'
Dbug = 'Off'     #Debugging - use On or Off  

# ===================================================================================================================
# Dynamic query seperation: CORE
# USE 'CI' for CORE INT, USE 'CC' for CORE CHAR, USE 'CV' for VARCHAR, USE 'CNV' for NVARCHAR, USE 'CN' for NCHAR 
# ===================================================================================================================
file04 = 'mmDynamicCI.sql'
file05 = 'mmDynamicNCI.sql'
file06 = 'mmReport.csv'
# ===================================================================================================================
file12 = 'mmMerge.csv'
# ===================================================================================================================

#Creating a report file to load todays data into at the specified location 
with open(os.path.join(pathMS, file04), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05), 'w') as fp: 
    pass

#Min (INT) Core - Create Dynamic SQL
for x in dbListC:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql = open(os.path.join(pathMS, 'mmCI.sql'))
        df = pd.read_sql_query(sql.read(), conn)

        with open(os.path.join(pathMS, file04), 'a') as f:
            df.to_string(f, header=None, index=False)

        sql.close()
        conn.close()
    finally:
        continue

# ===================================================================================================================
#  MinMax (INT) Core Processing 
# ===================================================================================================================
#Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicCI.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
now01 = datetime.datetime.now()
print('6 Processing Dynamic Min-Max C INT SQL: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

for sqlCmd in sqlFile.split(';'):
    for x in dbListC:
        try:
            conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))
            cursor = conn.cursor()
            cursor.execute(sqlCmd)
            row = cursor.fetchone()
            if row == None:
                break
            df = pd.DataFrame([row])
            #if df == None:
                #break
            with open(os.path.join(pathMR, file06), 'a') as f:
                df.to_csv(f, header=None, index=False, line_terminator='\n')
            with open(os.path.join(pathMR, file10), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            with open(os.path.join(pathMR, file12), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            #Close cursor to database
            cursor.close()
            #Close connection to database
            conn.close()
        except:
            continue
        finally:
            continue

now01 = datetime.datetime.now()
print('7 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")
# ===================================================================================================================
#  MinMax (INT) NonCore - Create Dynamic SQL 
# ===================================================================================================================
for x in dbListN:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql = open(os.path.join(pathMS, 'mmNCI.sql'))
        df = pd.read_sql_query(sql.read(), conn)

        with open(os.path.join(pathMS, file05), 'a') as f:
            df.to_string(f, header=None, index=False)

        sql.close()
        conn.close()
    finally:
        continue

# ===================================================================================================================
#  MinMax (INT) Non-Core Processing 
# ===================================================================================================================
#Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicNCI.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
now01 = datetime.datetime.now()
print('8 Processing Dynamic Min-Max NC INT SQL: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

for sqlCmd in sqlFile.split(';'):
    for x in dbListN:
        try:
            conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))
            cursor = conn.cursor()
            cursor.execute(sqlCmd)
            row = cursor.fetchone()
            if row == None:
                break
            df = pd.DataFrame([row])
            #if df == None:
                #break
            with open(os.path.join(pathMR, file06), 'a') as f:
                df.to_csv(f, header=None, index=False, line_terminator='\n')
            with open(os.path.join(pathMR, file11), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            with open(os.path.join(pathMR, file12), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            #Close cursor to database
            cursor.close()
            #Close connection to database
            conn.close()
        except:
            continue
        finally:
            continue

# ===================================================================================================================
now01 = datetime.datetime.now()
print('9 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('10 **** Connection Closed - Report Generated - Program Completed ****')

6 Processing Dynamic Min-Max C INT SQL:  2021-05-25 09:12:33
7 Processing Complete:  2021-05-25 09:12:55
8 Processing Dynamic Min-Max NC INT SQL:  2021-05-25 09:13:15
9 Processing Complete:  2021-05-25 09:31:30
10 **** Connection Closed - Report Generated - Program Completed ****


In [6]:
# Core Type 
# Variables: 
# ===================================================================================================================
#  Dynamic query seperation: CORE
#  USE 'CI' for INT, USE 'CC' for CHAR, USE 'CV' for VARCHAR, USE 'CNV' for NVARCHAR, USE 'CN' for NCHAR 
# ===================================================================================================================
file04 = 'mmDynamicCC.sql'
file05 = 'mmDynamicNCC.sql'
file06 = 'mmReport.csv'
# ===================================================================================================================
file12 = 'mmMerge.csv'
# ===================================================================================================================

#Creating a report file to load todays data into at the specified location 
with open(os.path.join(pathMS, file04), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05), 'w') as fp: 
    pass
# ===================================================================================================================
#  MinMax (CHAR) Core - Create Dynamic SQL
# ===================================================================================================================
for x in dbListC:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql = open(os.path.join(pathMS, 'mmCC.sql'))
        df = pd.read_sql_query(sql.read(), conn)

        with open(os.path.join(pathMS, file04), 'a') as f:
            df.to_string(f, header=None, index=False)

        sql.close()
        conn.close()
    finally:
        continue

# ===================================================================================================================
#  MinMax (CHAR) Core Processing 
# ===================================================================================================================
#Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicCC.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
now01 = datetime.datetime.now()
print('11 Processing Dynamic Min-Max C CHAR SQL: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

for sqlCmd in sqlFile.split(';'):
    for x in dbListC:
        try:
            conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))
            cursor = conn.cursor()
            cursor.execute(sqlCmd)
            row = cursor.fetchone()
            if row == None:
                break
            df = pd.DataFrame([row])
            #if df == None:
                #break
            with open(os.path.join(pathMR, file06), 'a') as f:
                df.to_csv(f, header=None, index=False, line_terminator='\n')
            with open(os.path.join(pathMR, file10), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            with open(os.path.join(pathMR, file12), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            #Close cursor to database
            cursor.close()
            #Close connection to database
            conn.close()
        except:
            continue
        finally:
            continue

now01 = datetime.datetime.now()
print('12 Processing Dynamic Min-Max C CHAR SQL Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")
# ===================================================================================================================
#  MinMax (CHAR) Core - Create Dynamic SQL 
# ===================================================================================================================
for x in dbListN:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql = open(os.path.join(pathMS, 'mmNCC.sql'))
        df = pd.read_sql_query(sql.read(), conn)

        with open(os.path.join(pathMS, file05), 'a') as f:
            df.to_string(f, header=None, index=False)

        sql.close()
        conn.close()
    finally:
        continue

# ===================================================================================================================
#  MinMax (CHAR) NonCore Processing 
# ===================================================================================================================
#Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicNCC.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
now01 = datetime.datetime.now()
print('13 Processing Dynamic Min-Max SQL: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

for sqlCmd in sqlFile.split(';'):
    for x in dbListN:
        try:
            conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))
            cursor = conn.cursor()
            cursor.execute(sqlCmd)
            row = cursor.fetchone()
            if row == None:
                break
            df = pd.DataFrame([row])
            #if df == None:
                #break
            with open(os.path.join(pathMR, file06), 'a') as f:
                df.to_csv(f, header=None, index=False, line_terminator='\n')
            with open(os.path.join(pathMR, file11), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            with open(os.path.join(pathMR, file12), 'a') as f:
                df.to_csv(f, header=None, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            #Close cursor to database
            cursor.close()
            #Close connection to database
            conn.close()
        except:
            continue
        finally:
            continue

# ===================================================================================================================
now01 = datetime.datetime.now()
print('14 Processing Dynamic Min-Max NC CHAR SQL Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('15 **** Connection Closed - Report Generated - Program Completed ****')

11 Processing Dynamic Min-Max C CHAR SQL:  2021-05-25 09:47:33
12 Processing Dynamic Min-Max C CHAR SQL Complete:  2021-05-25 09:47:33
13 Processing Dynamic Min-Max SQL:  2021-05-25 09:47:49
14 Processing Dynamic Min-Max NC CHAR SQL Complete:  2021-05-25 09:47:49
5 **** Connection Closed - Report Generated - Program Completed ****


In [7]:
print('16 **** Min-Max Date data generated for reporting - This Py Notebook Program has Completed ****')

16 **** Min-Max Date data generated for reporting - This Py Notebook Program has Completed ****


s01 = open(os.path.join(pathMR, 'mmReport.csv'))
sqlFile15 = s01.read()
s01.close()

#Data Prep step: split and strip special characters and attempt to get white space out of left pad
#Then, once dataset is ready ingest as dataframe and update the greg.dbo tables used for MinMax Date tracking

s = open(os.path.join(pathMR, 'mmMergeC.csv'),'r').read()

chars = ('$','%','^','*','(',')','\'') # etc
for c in chars:
  s = ' '.join( s.split(c) ).lstrip()

with open(os.path.join(pathMR, 'mmMergeC_new.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

now01 = datetime.datetime.now()
print('DataPrep C Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

s = open(os.path.join(pathMS, 'mmMergeN.csv','r').read()

chars = ('$','%','^','*','(',')','\'') # etc
for c in chars:
  s = ' '.join( s.split(c) ).lstrip()

with open(os.path.join(pathMR, 'mmMergeN_new.csv'), 'w') as f:
    df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')

now01 = datetime.datetime.now()
print('DataPrep C Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))